<a href="https://colab.research.google.com/github/cagBRT/PySpark/blob/master/Distrib_vs_Single_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we compare doing a task in a distributed environment and on a single CPU (single computer)

In [24]:
!git clone https://github.com/cagBRT/PySpark.git

fatal: destination path 'PySpark' already exists and is not an empty directory.




---



---



# Setup Spark<br>

If using DataBricks this section will be different

In [25]:
!pip install pyspark

In [26]:
#Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
#getOrCreate gets or creates a session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [27]:
import urllib.request
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [28]:
spark = (
    SparkSession.builder
        .appName('learn')
        .config('spark.driver.memory', '8g')
        .master('local[4]',)
        .config('spark.sql.execution.arrow.pyspark.enabled', True)
        .config('spark.sql.execution.arrow.pyspark.fallback.enabled', False)
        .getOrCreate()
)

In [29]:
sc=spark.sparkContext.getOrCreate()



---



---



We will now get all the works of Shakespeare (a very large file) and perform count and sort tasks.

# Get all of Shakespeare’s works

In [30]:
import time

**Let's find out how long it takes a distributed computer to perform the task of counting all the words in Shakespeare's works**.

In [31]:
start_time = time.time()
#Count the number of words in all of Shakespear's works
Words=sc.textFile("/content/PySpark/shakespeare.txt")
WordsCount=Words.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
WordsCount.count()

--- 0.06805062294006348 seconds ---


1418390

**Let's find out how long it takes a *single computer* to perform the task of counting all the words in Shakespeare's works**.

In [32]:
start_time = time.time()
f= open("/content/PySpark/shakespeare.txt", "r")
words_shakes= f.read()
f.close()
word_shakes_python=words_shakes.split(" ")
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(word_shakes_python)

--- 0.1206052303314209 seconds ---


1293935

In [33]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.0682218074798584
single compute time:  0.12168717384338379


In [34]:
start_time = time.time()
#Count the number of distinct words
DistinctWordsCount=WordsCount.reduceByKey(lambda a,b: a+b)
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
DistinctWordsCount.count()

--- 0.019227027893066406 seconds ---


67506

In [35]:
start_time = time.time()
words_unique=set(word_shakes_python)
print("--- %s seconds ---" % (time.time() - start_time))
sc_time=time.time() - start_time
len(words_unique)

--- 0.07577276229858398 seconds ---


86196

In [36]:
print("distributed time:    ",dc_time )
print("single compute time: ", sc_time)

distributed time:     0.019388914108276367
single compute time:  0.08773207664489746


**How long for counting distinct words?**

In [37]:
start_time = time.time()
#Sort the words by most-to-least words
SortedWordsCount=DistinctWordsCount.map(lambda a: (a[1], a[0])).sortByKey()
print("--- %s seconds ---" % (time.time() - start_time))
dc_time=time.time() - start_time
#print most frequent 20 words
SortedWordsCount.top(20)

--- 0.980170726776123 seconds ---


[(517065, ''),
 (23242, 'the'),
 (19540, 'I'),
 (18297, 'and'),
 (15623, 'to'),
 (15544, 'of'),
 (12532, 'a'),
 (10824, 'my'),
 (9576, 'in'),
 (9081, 'you'),
 (7851, 'is'),
 (7531, 'that'),
 (7068, 'And'),
 (6948, 'not'),
 (6722, 'with'),
 (6218, 'his'),
 (6009, 'your'),
 (6002, 'be'),
 (5616, 'for'),
 (5236, 'have')]

In [15]:
#start_time = time.time()
#counted_words=[]
#words_unique=list(words_unique)
#for i in range(len(words_unique)):
#  tuples= (word_shakes_python.count(words_unique[i]), words_unique[i])
#  counted_words.append(tuples)
#print("--- %s seconds ---" % (time.time() - start_time))
#sc_time= time.time() - start_time

In [17]:
#print("distributed time:    ",dc_time )
#print("single compute time: ", sc_time)

distributed time:     2.008025884628296<br>
single compute time:  1706.7746877670288